In [15]:
#importing required libraries
import glob
from imageai.Detection import VideoObjectDetection
import natsort
import numpy as np
import os
import sys
import numpy as np
import cv2
import glob
from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, nan, sqrt
import natsort
import pandas as pd
import random
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV
import sys
import cv2
from random import randint

In [16]:
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

def createTrackerByName(trackerType):
    
  # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
  
    elif trackerType == trackerTypes[1]: 
        tracker = cv2.TrackerMIL_create()
   
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
  
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
   
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
    
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
            
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
        
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
        
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)
     
    return tracker

In [17]:
def get_speed_from_frame(video, trackerType, boxes_list, frame_no):

    '''
    Detects speed of each vehicle in the frame and returns and a list 
    containing these speeds

    Parameters:
    videoPath(string) : path of the video
    trackerType(string) : the openCV tracker type to be used  
    boxes_list(list) : list containing bounding boxes for all detected vehicles in the particular frame
    frame_no : frame no of the video for which you require directions

    '''
    no_previous_frames = 5
    frame_buffer = []
    #cv2.namedWindow(video, cv2.WINDOW_NORMAL)
    #cv2.resizeWindow(video, height=650, width=1156)
    cap = cv2.VideoCapture(video)

    for i in range(frame_no - no_previous_frames, frame_no + 1):

        cap.set(1,i)
        ret, frame = cap.read()
        frame_buffer.append(frame)

    multiTracker_back = cv2.MultiTracker_create()
    #multiTracker = cv2.MultiTracker_create()
    n = len(boxes_list)
    speeds = [0]*n
    x1 = [0]*n  #x coordinates of vehicles in frame_no
    y1 = [0]*n #y coordinates of vehicles in frame_no
    x2 = [0]*n  #x coordinates of vehicles in frame_no - no_previous_frames
    y2 = [0]*n #y coordinates of vehicles in frame_no - no_previous_frames

    frame = frame_buffer.pop()
    for i in range(n):
        box = boxes_list[i]
        x1[i] = (box[0]+2*box[2])/2
        y1[i] = (box[1]+2*box[3])/2
        multiTracker_back.add(createTrackerByName(trackerType), frame, box)
        #all the vehicles from boxes_list added in the traker instance
        #applying vehicle tracking in backward direction
    for j in range(no_previous_frames):
        frame = frame_buffer.pop()
        (success_back, boxes_back) = multiTracker_back.update(frame)

    for i in range(n):
        b = boxes_back[i]
        x2[i] = (b[0]+2*b[2])/2
        y2[i] = (b[1]+2*b[3])/2
        x_d = x2[i] - x1[i]
        y_d = y2[i] - y1[i]
        dist = (x_d**2 + y_d**2)**0.5
        speeds[i] = dist

    return np.array(speeds)


In [ ]:
def get_speed_from_video(video, boxes_list):

    


In [18]:
vname = "C:/RoadCrossingAssistant/Data/Videos/video1.MOV"
fname = "C:/RoadCrossingAssistant/Data/Arrays_RetinaNet/array1.npy"

bounding_boxes = np.load(fname, allow_pickle=True) #loading the numpy array containing all detected vehicles
no_frames = bounding_boxes.shape[0]

frame_no = 35
temp = bounding_boxes[frame_no]
boxes_list = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in temp]

print(get_speed_from_frame(vname, "KCF", boxes_list, frame_no))


[69.02897942 69.          2.23606798 22.56102835 15.03329638  7.
 65.37583651  0.        ]


In [4]:
vname = "C:/RoadCrossingAssistant/Data/Videos/video1.MOV"

cv2.namedWindow('video', cv2.WINDOW_NORMAL)
cv2.resizeWindow('video', height=650, width=1156)

cap = cv2.VideoCapture(vname) #video_name is the video being called
cap.set(1,25); # Where frame_no is the frame you want
ret, frame = cap.read() # Read the frame
cv2.imshow('video', frame) # show frame on window
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
l = [1,2,3,4,5]
x = l.pop()
print(x)
print(l)

5
[1, 2, 3, 4]
